# Test the MPPCA algorithm on the roof classification problem

In [1]:
from PIL import Image
import csv
import collections
import numpy as np
import mppca
import time
import math
import pickle

from matplotlib import pyplot as plt
%matplotlib inline

## Fetch labels

In [2]:
orientations = load_orientations()

In [4]:
collections.Counter(orientations.values())

Counter({'': 22324, '1': 6114, '2': 3119, '3': 1538, '4': 3333})

In [5]:
train_ids, val_ids, test_ids, train_labels, val_labels, test_labels = split_set(orientations)

## Train on MPPCA

In [49]:
def train(train_data, K, p, q):
    N, d = train_data.shape
    
    pi = np.zeros(p*K)
    mu = np.zeros((p*K, d))
    W = np.zeros((p*K, d, q))
    sigma2 = np.zeros(p*K)
    cluster_labels = np.zeros(p*K, dtype=np.int32)

    for i in range(K):
        label = i+1
        X = train_data[train_labels == label]

        pi_i, mu_i, W_i, sigma2_i, clusters = mppca.initialization_kmeans(X, p, q)

        t = time.time()
        pi_i, mu_i, W_i, sigma2_i, R, L, sigma2hist = mppca.mppca_gem(
            X, pi_i, mu_i, W_i, sigma2_i, niter)
        print('GEM took %f seconds.'%(time.time()-t))

        pi[p*i:p*(i+1)] = pi_i
        mu[p*i:p*(i+1), :] = mu_i
        W[p*i:p*(i+1), :, :] = W_i
        sigma2[p*i:p*(i+1)] = sigma2_i
        cluster_labels[p*i:p*(i+1)] = label
    
    return pi, mu, W, sigma2, cluster_labels

In [50]:
def test(test_data, test_labels, pi, mu, W, sigma2, cluster_labels):
    R = mppca.mppca_predict(test_data, pi, mu, W, sigma2)

    clusters = R.argmax(axis=1)
    predicted_classes = cluster_labels[clusters]

    error_rate = (test_labels!=predicted_classes).sum()/len(test_labels)
    
    return error_rate

## Vizualization

In [ ]:
i = 9
# center of the i-th cluster
plt.imshow(mu[i, :].reshape((height, width)))
print('label=%d'%cluster_labels[i])

In [ ]:
# Plot the 1st principal component of the i-th cluster
plt.imshow(W[i, :, 0].reshape((height, width)))

## Parameter selection

In [51]:
niter = 100
K = 2

# exp1
#color = False
#l_values = [10, 12, 15, 18, 20, 25, 30]
#p_values = range(1, 10)
#q_values = range(1, 7)

# exp2
#color = False
#l_values = [5, 6, 7, 8, 9, 10]
#p_values = range(1, 15)
#q_values = range(1, 15)

# exp3
color = True
l_values = [5, 6, 7, 8, 9, 10]
p_values = range(1, 15)
q_values = range(1, 15)

In [53]:
error_rates = np.ones((max(l_values)+1, max(p_values)+1, max(q_values)+1))

for l in l_values:
    
    print('l=%d'%l)
    train_data, val_data, test_data = load_all_data(train_ids, val_ids, test_ids, l, color)
    
    for p in p_values: # Number of expected clusters (per class)
        for q in q_values: # Dimension of the subspaces
            
            pi, mu, W, sigma2, cluster_labels = train(train_data, K, p, q)
            error_rate = test(val_data, val_labels, pi, mu, W, sigma2, cluster_labels)
            error_rates[l, p, q] = error_rate
            print('K=%d; p=%d; q=%d; e=%f'%(K, p, q, error_rate))

l=5
....................................................................................................GEM took 0.605108 seconds.
....................................................................................................GEM took 0.586211 seconds.
K=2; p=1; q=1; e=0.211000
....................................................................................................GEM took 0.614505 seconds.
....................................................................................................GEM took 0.624283 seconds.
K=2; p=1; q=2; e=0.210000
....................................................................................................GEM took 0.621250 seconds.
....................................................................................................GEM took 0.610230 seconds.
K=2; p=1; q=3; e=0.178000
....................................................................................................GEM took 0.610693 seconds.
.............................

/data/projects/opensolarmap/MPPCA/mppca.py:104: RuntimeWarning: divide by zero encountered in true_divide
  ) / sigma2[c]
/data/projects/opensolarmap/MPPCA/mppca.py:115: RuntimeWarning: invalid value encountered in log
  - 0.5*(deviation_from_center * np.dot(deviation_from_center, Cinv[c, :, :].T)).sum(1)
/data/projects/opensolarmap/MPPCA/mppca.py:115: RuntimeWarning: divide by zero encountered in log
  - 0.5*(deviation_from_center * np.dot(deviation_from_center, Cinv[c, :, :].T)).sum(1)


In [60]:
flat_argmin = np.argmin(error_rates)
l_opt, p_opt, q_opt = np.unravel_index(flat_argmin, error_rates.shape)
np.min(error_rates), l_opt, p_opt, q_opt

(0.11600000000000001, 6, 9, 13)

In [61]:
train_data, val_data, test_data = load_all_data(l=l_opt, color=color)
pi, mu, W, sigma2, cluster_labels = train(train_data, K=2, p=p_opt, q=q_opt)
test(test_data, test_labels, pi, mu, W, sigma2, cluster_labels)

....................................................................................................GEM took 10.249007 seconds.
....................................................................................................GEM took 10.115154 seconds.


0.159

In [58]:
error_rates_exp3 = error_rates
pickle.dump(error_rates_exp3, open('exp3.pkl', 'wb'))

In [59]:
error_rates = pickle.load(open('exp2.pkl', 'rb'))